## This Jupyter Notebook is used to convert raw .mib SPED files from to the more practical .hspy file format 

In [ ]:
%matplotlib qt
import hyperspy.api as hs
import pyxem as pxm
from pathlib import Path
from math import atan, sqrt
import os

In [ ]:
def wavelength(V, 
            m0=9.1093837015*1e-31, 
            e=1.60217662*1e-19, 
            h=6.62607004*1e-34 , 
            c=299792458):
    """
    Return the wavelength of an accelerated electron in [Å]
    
    Arguments
    ---------
    V : float, Acceleration voltage of electrons [kV]
    m0 : float, Rest mass of electron [kg]
    e : float, Elementary charge of electron [C]
    h : float, Planck' constant [m^2 kg/s]
    c : float, Speed of light in vacuum [m/s]
    """
    V = V*1E3
    return h / sqrt( 2 * m0 * e * V * ( 1.0 + ( e*V / ( 2*m0*c**2 ) ) ) ) * 1E10

def load_hdr(filename):
        """load a header file"""
        filename=Path(filename)
        hdr_content = dict()
        if filename.exists() and filename.suffix == '.hdr':
            with filename.open('r') as hdrfile:
                lines = hdrfile.readlines()
                for lineno, line in enumerate(lines):
                    if 0 < lineno < len(lines)-1:
                        field, value = line.split(':', maxsplit=1)
                        field = field.strip()
                        value = value.strip()
                        hdr_content[field] = value
        else:
            raise FileNotFoundError(f'HDR file "{filename.absolute()}" \
                                    does not exist or is not a valid .hdr file.')
        return hdr_content

In [ ]:
data_path = Path('Path_to_your/file.mib')
beam_energy = 200 #Beam energy in keV
cameralength = 22.95 #Camera length in cm
precession_angle= 1 #Precession angle in degrees
precession_frequency=100 #Precession frequency in Hz
exposure_time = 10 #exposure time in ms

step_size = (13.9, 13.9) #Scan step sizes in (x, y)
scan_units = 'nm'

signal = pxm.load_mib(str(data_path), flip=True, reshape=False) #Flip True and reshape False needed to load correcly

signal.set_experimental_parameters(beam_energy=beam_energy, 
                                    camera_length=cameralength, 
                                    rocking_angle=precession_angle, 
                                    rocking_frequency=precession_frequency, 
                                    exposure_time=exposure_time)

hdr_dict = load_hdr(data_path.with_suffix('.hdr'))

signal.metadata.add_dictionary({'Acquisition_instrument': {'Merlin_hdr': hdr_dict}})
signal.original_metadata.add_dictionary({'HDR': hdr_dict})


In [ ]:
scan_shape = (128,128) #Enter scan shape here
scan_chunks = (32, 32) #Defines how stored in memory

detector_chunks = (32, 32)
detector_shape = (256, 256) #Detector on 2100F is always 256,256

data = signal.data
data = data.reshape(scan_shape + detector_shape)
data = data.rechunk(scan_chunks + detector_chunks)

s = pxm.signals.LazyElectronDiffraction2D(data[:,:,:,:])

s.metadata.add_dictionary(signal.metadata.as_dictionary())
s.original_metadata.add_dictionary(signal.original_metadata.as_dictionary())

s.set_signal_type("electron_diffraction")

s.data

In [ ]:
#plot to check it looks ok
# If wrong, try to swap X and Y shape in scan_shape in cell above

s.plot()

In [ ]:
make_subdir = True

pixel_size = 55E-6 #Camera pixel size in m. Merlin pixel size is 55 um
names = ['x', 'y', 'kx', 'ky']

#Calculate pixel diffraction scales in k-space
dx = abs(atan(pixel_size * s.axes_manager[2].size / \
s.metadata.Acquisition_instrument.TEM.Detector.Diffraction.camera_length) / \
wavelength(s.metadata.Acquisition_instrument.TEM.beam_energy) / s.axes_manager[2].size)

dy = abs(atan(pixel_size * s.axes_manager[3].size / \
s.metadata.Acquisition_instrument.TEM.Detector.Diffraction.camera_length) / \
wavelength(s.metadata.Acquisition_instrument.TEM.beam_energy) / s.axes_manager[3].size)

scales = [step_size[0], step_size[1], dx, dy]
offsets = [0, 0, -dx*s.axes_manager[2].size/2, -dy*s.axes_manager[2].size/2]
units = [scan_units, scan_units, '$Å^{-1}$', '$Å^{-1}$']
for ax_no, (name, scale, offset, unit) in enumerate(zip(names, scales, offsets, units)):
    s.axes_manager[ax_no].name = name
    s.axes_manager[ax_no].scale = scale
    s.axes_manager[ax_no].offset = offset
    s.axes_manager[ax_no].units = unit

print(s.axes_manager)


if make_subdir:
    output_directory = Path(f'{data_path.parent}/{data_path.stem}')
    os.makedirs(output_directory.absolute(), exist_ok=True)
    output_path = output_directory / data_path.with_suffix('.hspy').name #or .zspy
else:
    output_path = data_path.with_suffix('.hspy') #or .zspy
s.save(str(output_path))   